In [1]:
import my_toolbox as tb
import numpy as np
import calibration as calib
import main
import importlib

importlib.reload(main)

main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
age_type_input_path = main_path + "input/50p_age_moms/"
trans_path = age_type_input_path + "MH_trans_by_MH_clust_age.csv"  
# my_grid = np.array([5.0, 10.0, 15.0, 20.0])
my_grid = np.linspace(0.5, 20, 10)
my_grid = np.log(my_grid)
myPars = main.pars_factory(main_path, my_lab_fe_grid = my_grid, H_trans_path = trans_path) 




Using health transition matrix from: C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/input/50p_age_moms/MH_trans_by_MH_clust_age.csv


In [13]:
importlib.reload(calib)
target_fold_path = myPars.path + "/input/50p_age_moms/"

mu_mom_targ = 2.2
sigma_mom_targ = .299
w1_mom_targ = calib.get_w1_targ(myPars, target_fold_path)
w2_mom_targ = calib.get_w2_targ(myPars, target_fold_path)

# define lambda function to minimize
mu = 2.0
sigma = .3
w1 = 1.0
w2 = -0.2
myPars.lab_fe_tauch_mu = mu
myPars.lab_fe_tauch_sigma = sigma
myPars.set_w1(w1)
myPars.set_w2(w2)

mu_mom_err = lambda new_mu: calib.w0_mu_mom_giv_mu(myPars, new_mu) - mu_mom_targ
sig_mom_err = lambda new_sig: calib.w0_sigma_mom_giv_sigma(myPars, new_sig) - sigma_mom_targ
w1_mom_err = lambda new_w1: calib.w1_moment_giv_w1(myPars, new_w1) - w1_mom_targ 
w2_mom_err = lambda new_w2: calib.w2_moment_giv_w2(myPars, new_w2) - w2_mom_targ

calib_mu = -999  # initialize
mu_tol = 0.001
calib_sigma = -999
sig_tol = 0.001
calib_w1 = -999
w1_tol = 0.001
calib_w2 = -999
w2_tol = 0.001

max_iter = 30

w1_iters = 0
w1_max = .1
w1_min = 0.0
while(abs(w1_mom_err(myPars.wage_coeff_grid[0,1])) > w1_tol and w1_iters < max_iter):
    print(f"Starting w1 iteration {w1_iters}")
    print(f"Current w1: {myPars.wage_coeff_grid[0,1]}, w1 moment: {calib.w1_moment(myPars)}, target w1 moment: {w1_mom_targ}, w1 error: {w1_mom_err(myPars.wage_coeff_grid[0,1])}")

    w2_iters = 0
    w2_max = 0.0
    w2_min = -0.1
    while(abs(w2_mom_err(myPars.wage_coeff_grid[0,2])) > w2_tol and w2_iters < max_iter):
        # print(f"Starting w2 iteration {w2_iters}")
        # print(f"Current w2: {myPars.wage_coeff_grid[0,2]}, w2 moment: {calib.w2_moment(myPars)}, target w2 moment: {w2_mom_targ}, w2 error: {w2_mom_err(myPars.wage_coeff_grid[0,2])}")
        # print(f"Current w2: {myPars.wage_coeff_grid[0,2]}")

        mu_iters = 0
        mu_max = 5.0
        mu_min = 0.0
        while abs(mu_mom_err(myPars.lab_fe_tauch_mu)) > mu_tol and mu_iters < max_iter:
            # print(f"Starting mu iteration {mu_iters}")
            # print(f"Current mu: {myPars.lab_fe_tauch_mu}")

            sig_iters = 0
            sig_max = 1.0
            sig_min = 0.001
            while abs(sig_mom_err(myPars.lab_fe_tauch_sigma)) > sig_tol and sig_iters < max_iter:
                # print(f"Starting sigma iteration {sig_iters}")
                # print(f"Current sigma: {myPars.lab_fe_tauch_sigma}")
                sig_mid_pt = (sig_max + sig_min) / 2
                sig_mom_err_mid = sig_mom_err(sig_mid_pt)
                if sig_mom_err_mid > 0:
                    sig_max = sig_mid_pt
                else:
                    sig_min = sig_mid_pt
                sig_iters += 1
                calib_sigma = sig_mid_pt

            mu_mid_pt = (mu_max + mu_min) / 2
            mu_mom_err_mid = mu_mom_err(mu_mid_pt)
            if mu_mom_err_mid > 0:
                mu_max = mu_mid_pt
            else:
                mu_min = mu_mid_pt
            mu_iters += 1
            calib_mu = mu_mid_pt

        w2_mid_pt = (w2_max + w2_min) / 2
        w2_mom_err_mid = w2_mom_err(w2_mid_pt)
        if w2_mom_err_mid < 0: # since decreasing w2 increases decay may want to adjust moment functions to keep consistent
            w2_max = w2_mid_pt
        else:
            w2_min = w2_mid_pt
        w2_iters += 1
        calib_w2 = w2_mid_pt

    w1_mid_pt = (w1_max + w1_min) / 2
    w1_mom_err_mid = w1_mom_err(w1_mid_pt)
    if w1_mom_err_mid > 0:
        w1_max = w1_mid_pt
    else:
        w1_min = w1_mid_pt
    w1_iters += 1
    calib_w1 = w1_mid_pt

print(f"After {w1_iters} iterations")
print(f"Calibrated w2: {calib_w2}, myPars w2: {myPars.wage_coeff_grid[0,2]}, model w2 moment: {calib.w2_moment(myPars)}, target w2 moment: {w2_mom_targ}")
print(f"Calibrated w1: {calib_w1}, myPars w1: {myPars.wage_coeff_grid[0,1]}, model w1 moment: {calib.w1_moment(myPars)}, target w1 moment: {w1_mom_targ}")
print(f"Calibrated mu: {calib_mu}, myPars mu: {myPars.lab_fe_tauch_mu}, model mu moment: {calib.w0_mu_moment(myPars)}, target mu moment: {mu_mom_targ}")
print(f"Calibrated sigma: {calib_sigma}, myPars sigma: {myPars.lab_fe_tauch_sigma}, model sigma moment: {calib.w0_sigma_moment(myPars)}, target sigma moment: {sigma_mom_targ}")
print()


Starting w1 iteration 0
Current w1: 1.0, w1 moment: 1.2010152069499043, target w1 moment: 0.2915204, w1 error: 0.9094948069499043
Starting w1 iteration 1
Current w1: 0.05, w1 moment: 0.051012189692366317, target w1 moment: 0.2915204, w1 error: -0.2405082103076337
Starting w1 iteration 2
Current w1: 0.07500000000000001, w1 moment: 1.7627621951628747, target w1 moment: 0.2915204, w1 error: 1.4712417951628747
Starting w1 iteration 3
Current w1: 0.0625, w1 moment: 0.8806633616694595, target w1 moment: 0.2915204, w1 error: 0.5891429616694595
Starting w1 iteration 4
Current w1: 0.05625, w1 moment: 0.8217034025004364, target w1 moment: 0.2915204, w1 error: 0.5301830025004364
Starting w1 iteration 5
Current w1: 0.053125000000000006, w1 moment: 0.7943166122931218, target w1 moment: 0.2915204, w1 error: 0.5027962122931218
Starting w1 iteration 6
Current w1: 0.051562500000000004, w1 moment: 0.780400597085908, target w1 moment: 0.2915204, w1 error: 0.488880197085908
Starting w1 iteration 7
Current

In [19]:
max_iter = 50
iters = 0

while abs(w1_mom_err(myPars.wage_coeff_grid[0,1])) > w1_tol and w2_mom_err(myPars.wage_coeff_grid[0,2]) > w2_tol and iters < max_iter:
    calib_w2 = tb.bisection_search(w2_mom_err, -1.0, 0.0, tol = w2_tol, max_iter = max_iter)
    myPars.set_w2(calib_w2)
    print(f"Uncalibrated w1: {myPars.wage_coeff_grid[0,1]}, model w1 moment: {calib.w1_moment(myPars)}, target w1 moment: {w1_mom_targ}")
    print(f"Calibrated w2: {calib_w2}, myPars w2: {myPars.wage_coeff_grid[0,2]}, model w2 moment: {calib.w2_moment(myPars)}, target w2 moment: {w2_mom_targ}")

    calib_w1 = tb.bisection_search(w1_mom_err, 0.0, 10.0, tol = w1_tol, max_iter = max_iter)
    myPars.set_w1(calib_w1)
    print(f"Calibrated w1: {calib_w1}, myPars w1: {myPars.wage_coeff_grid[0,1]}, model w1 moment: {calib.w1_moment(myPars)}, target w1 moment: {w1_mom_targ}")
    print(f"Uncalibrated w2: {myPars.wage_coeff_grid[0,2]}, model w2 moment: {calib.w2_moment(myPars)}, target w2 moment: {w2_mom_targ}")

In [4]:
calib_mu = myPars.lab_fe_tauch_mu
calib_sigma = myPars.lab_fe_tauch_sigma
calib_sigma = 1.0

tauch_weights, tauch_grid = tb.Taucheniid(calib_sigma, len(my_grid), Nsd = 3, mean = calib_mu, state_grid = my_grid)
print(f"Tauch weights: {tauch_weights}")
print(f"Tauchen grid: {tauch_grid}")

def tauch_mean(tauch_weights, tauch_grid):
    return np.sum(tauch_weights*tauch_grid)

def tauch_sd(tauch_weights, tauch_grid):
    return np.sqrt(np.sum(tauch_weights*(tauch_grid - tauch_mean(tauch_weights, tauch_grid))**2))

print(f"Calib_mu: {calib_mu}")
print(f"Tauch mean: {tauch_mean(tauch_weights, tauch_grid)}")
print(f"Calib_sigma: {calib_sigma}")
print(f"Tauch sd: {tauch_sd(tauch_weights, tauch_grid)}")

Tauch weights: [0.00446563 0.16896423 0.19124498 0.14505163 0.10591239 0.07801043
 0.0584664  0.04461724 0.03462425 0.16864282]
Tauchen grid: [-0.69314718  0.98082925  1.57553636  1.94591015  2.21557372  2.42774824
  2.60268969  2.75153531  2.88106937  2.99573227]
Calib_mu: 2.12646484375
Tauch mean: 2.0501463834146296
Calib_sigma: 1.0
Tauch sd: 0.7070908394205085
